In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.metrics import (roc_auc_score, roc_curve, auc, accuracy_score, classification_report, 
confusion_matrix, accuracy_score)

from sklearn.model_selection import (cross_val_score, cross_val_predict, train_test_split, 
StratifiedKFold, GridSearchCV)

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier

from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import (RandomForestClassifier)

from sklearn.pipeline import Pipeline

from textblob import TextBlob, Word

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/mbp15/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mbp15/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
'''
Here I used random forest classification to determine key features for predicting Data Scientist as a job versus
other Data job categories. I did not have much time to do this, so I did not optimise my keywords and just used
the simplified results to identify key different skills.
'''

df = pd.read_csv('./df_modelling', index_col = 0)

In [4]:
df

,job_title,job_location,advertiser,url,job_post_date,job_desc,pay_total2,jbd,salary
0,data scientist,Sydney,Motion Recruitment,www.seek.com.au/job/38274601?type=standout&sea...,23 Jan 2019,ABOUT QHR/eQ8 year of providing consulting...,120999.45,Normal Data Scientist,0.0
1,data science delivery lead,Sydney,Intellify,www.seek.com.au/job/38267034?type=standout&sea...,5 Feb 2019,About Intellify Intellify is a data science an...,165000.00,Senior Data Scientist,1.0
2,head of data science engineering & products,Sydney,FIRESOFT Consulting,www.seek.com.au/job/38248336?type=standard&sea...,1 Feb 2019,Head of Data Science Engineering & Products - ...,120999.45,Normal Data Scientist,0.0
3,machine learning engineer / scientist - optimi...,Sydney,Infopeople,www.seek.com.au/job/38178979?type=standard&sea...,23 Jan 2019,This role is a newly created role to work with...,160000.00,Senior Data Scientist,1.0
4,data analytics & data science team leader,Sydney,AUSTRALIAN FEDERAL POLICE,www.seek.com.au/job/38194317?type=standout&sea...,25 Jan 2019,The mission of the AFP is to provide dynamic a...,115046.50,Senior Data Scientist,0.0
5,senior data science executive,Sydney,The Nielsen Company (Australia) Pty Ltd,www.seek.com.au/job/38257609?type=standout&sea...,4 Feb 2019,Nielsen is a global performance management com...,140000.00,Senior Data Scientist,0.0
6,junior data scientist,Sydney,Motion Recruitment,www.seek.com.au/job/38108166?type=standout&sea...,15 Jan 2019,About the business and the role My client is ...,120999.45,Normal Data Scientist,0.0
7,data scientist with consultancy experience,Sydney,360HR Pty Ltd,www.seek.com.au/job/38261034?type=standard&sea...,4 Feb 2019,"In this role, you will be managing data from m...",120999.45,Normal Data Scientist,0.0
8,senior data science (loyalty) :: $170k + super...,Sydney,Correlate Resources,www.seek.com.au/job/38153321?type=standout&sea...,21 Jan 2019,Senior Data Scientist (Loyalty) :: $170K + Sup...,187000.00,Senior Data Scientist,1.0
9,senior data scientist,Sydney,Preacta Recruitment,www.seek.com.au/job/38265180?type=standard&sea...,5 Feb 2019,Currently working for one of Australia’s leadi...,140000.00,Senior Data Scientist,0.0


In [5]:
df.jbd.value_counts(normalize = True)

Normal Data Analyst             0.341653
Normal Business Intelligence    0.233500
Normal Data Engineer            0.158625
Senior Business Intelligence    0.074875
Senior Data Engineer            0.072102
Senior Data Analyst             0.067665
Normal Data Scientist           0.036606
Senior Data Scientist           0.014975
Name: jbd, dtype: float64

In [6]:
df.job_desc = df.job_desc.str.replace(',', '')
df.job_desc = df.job_desc.str.replace('$', '')
df.job_desc = df.job_desc.str.replace('-', '')
df.job_desc = df.job_desc.str.replace('’', '')
df.job_desc = df.job_desc.str.replace('/', '')
df.job_desc = df.job_desc.str.replace('+', '')
df.job_desc = df.job_desc.str.replace('&', '')
df.job_desc = df.job_desc.str.replace("'", '')
df.job_desc = df.job_desc.str.replace('@', '')
df.job_desc = df.job_desc.str.replace(';', '')
df.job_desc = df.job_desc.str.replace('%', '')
df.job_desc = df.job_desc.str.replace('•', '')
df.job_desc = df.job_desc.str.replace('!', '')
df.job_desc = df.job_desc.str.replace('#', '')
df.job_desc = df.job_desc.str.replace(':', '')
df.job_desc = df.job_desc.str.replace('(', '')
df.job_desc = df.job_desc.str.replace(')', '')
df.job_desc = df.job_desc.str.replace('.', '')

In [7]:
df['data science'] = 0

In [18]:
df.loc[df.jbd.str.contains('Scientist'), 'data science'] = 1

In [19]:
df

,job_title,job_location,advertiser,url,job_post_date,job_desc,pay_total2,jbd,salary,data science
0,data scientist,Sydney,Motion Recruitment,www.seek.com.au/job/38274601?type=standout&sea...,23 Jan 2019,ABOUT QHReQ8 year of providing consulting ...,120999.45,Normal Data Scientist,0.0,1
1,data science delivery lead,Sydney,Intellify,www.seek.com.au/job/38267034?type=standout&sea...,5 Feb 2019,About Intellify Intellify is a data science an...,165000.00,Senior Data Scientist,1.0,1
2,head of data science engineering & products,Sydney,FIRESOFT Consulting,www.seek.com.au/job/38248336?type=standard&sea...,1 Feb 2019,Head of Data Science Engineering Products Su...,120999.45,Normal Data Scientist,0.0,1
3,machine learning engineer / scientist - optimi...,Sydney,Infopeople,www.seek.com.au/job/38178979?type=standard&sea...,23 Jan 2019,This role is a newly created role to work with...,160000.00,Senior Data Scientist,1.0,1
4,data analytics & data science team leader,Sydney,AUSTRALIAN FEDERAL POLICE,www.seek.com.au/job/38194317?type=standout&sea...,25 Jan 2019,The mission of the AFP is to provide dynamic a...,115046.50,Senior Data Scientist,0.0,1
5,senior data science executive,Sydney,The Nielsen Company (Australia) Pty Ltd,www.seek.com.au/job/38257609?type=standout&sea...,4 Feb 2019,Nielsen is a global performance management com...,140000.00,Senior Data Scientist,0.0,1
6,junior data scientist,Sydney,Motion Recruitment,www.seek.com.au/job/38108166?type=standout&sea...,15 Jan 2019,About the business and the role My client is ...,120999.45,Normal Data Scientist,0.0,1
7,data scientist with consultancy experience,Sydney,360HR Pty Ltd,www.seek.com.au/job/38261034?type=standard&sea...,4 Feb 2019,In this role you will be managing data from mu...,120999.45,Normal Data Scientist,0.0,1
8,senior data science (loyalty) :: $170k + super...,Sydney,Correlate Resources,www.seek.com.au/job/38153321?type=standout&sea...,21 Jan 2019,Senior Data Scientist Loyalty 170K Super Bo...,187000.00,Senior Data Scientist,1.0,1
9,senior data scientist,Sydney,Preacta Recruitment,www.seek.com.au/job/38265180?type=standard&sea...,5 Feb 2019,Currently working for one of Australias leadin...,140000.00,Senior Data Scientist,0.0,1


In [20]:
df1 = df[df['data science'] != 0]
df2 = df[df['data science'] != 1]

In [21]:
df1

,job_title,job_location,advertiser,url,job_post_date,job_desc,pay_total2,jbd,salary,data science
0,data scientist,Sydney,Motion Recruitment,www.seek.com.au/job/38274601?type=standout&sea...,23 Jan 2019,ABOUT QHReQ8 year of providing consulting ...,120999.45,Normal Data Scientist,0.0,1
1,data science delivery lead,Sydney,Intellify,www.seek.com.au/job/38267034?type=standout&sea...,5 Feb 2019,About Intellify Intellify is a data science an...,165000.00,Senior Data Scientist,1.0,1
2,head of data science engineering & products,Sydney,FIRESOFT Consulting,www.seek.com.au/job/38248336?type=standard&sea...,1 Feb 2019,Head of Data Science Engineering Products Su...,120999.45,Normal Data Scientist,0.0,1
3,machine learning engineer / scientist - optimi...,Sydney,Infopeople,www.seek.com.au/job/38178979?type=standard&sea...,23 Jan 2019,This role is a newly created role to work with...,160000.00,Senior Data Scientist,1.0,1
4,data analytics & data science team leader,Sydney,AUSTRALIAN FEDERAL POLICE,www.seek.com.au/job/38194317?type=standout&sea...,25 Jan 2019,The mission of the AFP is to provide dynamic a...,115046.50,Senior Data Scientist,0.0,1
5,senior data science executive,Sydney,The Nielsen Company (Australia) Pty Ltd,www.seek.com.au/job/38257609?type=standout&sea...,4 Feb 2019,Nielsen is a global performance management com...,140000.00,Senior Data Scientist,0.0,1
6,junior data scientist,Sydney,Motion Recruitment,www.seek.com.au/job/38108166?type=standout&sea...,15 Jan 2019,About the business and the role My client is ...,120999.45,Normal Data Scientist,0.0,1
7,data scientist with consultancy experience,Sydney,360HR Pty Ltd,www.seek.com.au/job/38261034?type=standard&sea...,4 Feb 2019,In this role you will be managing data from mu...,120999.45,Normal Data Scientist,0.0,1
8,senior data science (loyalty) :: $170k + super...,Sydney,Correlate Resources,www.seek.com.au/job/38153321?type=standout&sea...,21 Jan 2019,Senior Data Scientist Loyalty 170K Super Bo...,187000.00,Senior Data Scientist,1.0,1
9,senior data scientist,Sydney,Preacta Recruitment,www.seek.com.au/job/38265180?type=standard&sea...,5 Feb 2019,Currently working for one of Australias leadin...,140000.00,Senior Data Scientist,0.0,1


In [22]:
df2

,job_title,job_location,advertiser,url,job_post_date,job_desc,pay_total2,jbd,salary,data science
18,senior business intelligence consultant,Sydney,AI Australia,www.seek.com.au/job/38225977?type=standout&sea...,30 Jan 2019,As a leading Australian owned Artificial Intel...,85000.00,Senior Business Intelligence,0.0,0
26,"consultant - data, analytics and machine learn...",Sydney,Alloc8,www.seek.com.au/job/38094512?type=standout&sea...,14 Jan 2019,Were proud to call this company a client and h...,150000.00,Senior Data Analyst,0.0,0
29,senior data engineer,Sydney,Drive Yello,www.seek.com.au/job/38091794?type=standout&sea...,14 Jan 2019,Welcome to Drive Yello Were a lastmile deliver...,135000.00,Senior Data Engineer,0.0,0
31,big data engineer x2,Sydney,M&T Resources,www.seek.com.au/job/38273080?type=standard&sea...,5 Feb 2019,My client is a major Australian Financial Serv...,125000.00,Normal Data Engineer,0.0,0
32,business analyst,Sydney,IQVIA,www.seek.com.au/job/38255623?type=standard&sea...,4 Feb 2019,Join us on our exciting journey IQVIA™ is The ...,118146.60,Normal Business Intelligence,0.0,0
33,credit risk analyst,Sydney,Huxley Associates,www.seek.com.au/job/38267995?type=standard&sea...,5 Feb 2019,Responsibilities Design and development of sc...,100000.00,Normal Data Analyst,0.0,0
36,16922-site sppt engineer 1,Sydney,IQVIA,www.seek.com.au/job/38240359?type=standard&sea...,1 Feb 2019,Join us on our exciting journey IQVIA™ is The ...,125000.00,Normal Data Engineer,0.0,0
40,big data engineer (contract) : sydney : 6 mont...,Sydney,Correlate Resources,www.seek.com.au/job/38277481?type=standout&sea...,17 Jan 2019,Big Data Engineer Initial 6 Month Contract ...,346750.00,Normal Data Engineer,1.0,0
42,big data engineer,Sydney,Hydrogen Group Pty Ltd,www.seek.com.au/job/38208957?type=standard&sea...,29 Jan 2019,Job Description As a key member of a Big Data ...,137500.00,Normal Data Engineer,1.0,0
46,junior digital marketing analyst (urgent),Sydney,T+O+M Executive,www.seek.com.au/job/38262679?type=standard&sea...,4 Feb 2019,Since embarking on a major customer transforma...,71500.00,Normal Data Analyst,0.0,0


In [23]:
X1 = df1['job_desc']

In [24]:
y1 = df1['jbd']

In [25]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.3, random_state = 42)

In [26]:
stop = stopwords.words('english')

In [27]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [28]:
model2 = Pipeline(
            steps = [('tfidf', TfidfVectorizer(tokenizer=LemmaTokenizer(), 
                                               ngram_range=(1,2),
                                               stop_words=stop, 
                                               max_features=150,
                                               max_df = 0.5,
                                               sublinear_tf = True)), 
                      ('gs', GridSearchCV(RandomForestClassifier(),
                                          param_grid={'n_estimators': [530, 535, 540],
                                                      'max_depth' : [15,20,25]},
                                                      cv=5,
                                                      refit=True))]
                  )

model2.fit(X1_train, y1_train)
y_pred = model2.predict(X1_test)

print(accuracy_score(y1_test, y_pred))
print("Number of features:", len(model2.steps[0][1].get_feature_names()))

# print(model.steps[0][1].get_feature_names())
# print(model.steps[1][1].feature_importances_)
# print(model.named_steps['rfc'].feature_importances_)


print(model2.steps[1][1].best_params_)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.8214285714285714
Number of features: 150
{'max_depth': 25, 'n_estimators': 530}


In [38]:
def RandomForest():
    model2 = Pipeline(
                steps = [('tfidf', TfidfVectorizer(tokenizer=LemmaTokenizer(), 
                                                   ngram_range=(1,2),
                                                   stop_words=stop, 
                                                   max_features=150,
                                                   max_df = 0.2,
                                                   sublinear_tf = True)), 
                          ('rfc', RandomForestClassifier(n_estimators=535,
                                                         max_depth=25,
                                                         random_state=42))]
                      )

    model2.fit(X1_train, y1_train)
    y_pred = model2.predict(X1_test)

    print(accuracy_score(y1_test, y_pred))
    print("Number of features:", len(model2.steps[0][1].get_feature_names()))

    # print(model.steps[0][1].get_feature_names())
    # print(model.steps[1][1].feature_importances_)
    # print(model.named_steps['rfc'].feature_importances_)



    feature_importances = pd.DataFrame(model2.steps[1][1].feature_importances_, model2.steps[0][1].get_feature_names()).reset_index()
    feature_importances.columns = ['feature', 'importance']
    features = feature_importances.sort_values('importance', ascending=False)
    print(features.sort_values('importance', ascending=False).head(20))

In [39]:
RandomForest()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.6785714285714286
Number of features: 150
         feature  importance
88       network    0.038146
16       benefit    0.034245
66       improve    0.032078
61        hadoop    0.029341
120  senior data    0.027552
19         brand    0.022374
36    developing    0.022175
24      computer    0.020192
9      algorithm    0.019572
98           phd    0.018445
85        method    0.018124
23   communicate    0.017234
125        spark    0.016793
110        range    0.016557
39     effective    0.016428
123      solving    0.015646
27    contribute    0.015111
106  programming    0.014691
126     standard    0.014361
144        world    0.013198


In [33]:
X2 = df2['job_desc']

In [34]:
y2 = df2['jbd']

In [35]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.3, random_state = 42)

In [40]:
def RandomForest2():
    model2 = Pipeline(
                steps = [('tfidf', TfidfVectorizer(tokenizer=LemmaTokenizer(), 
                                                   ngram_range=(1,2),
                                                   stop_words=stop, 
                                                   max_features=150,
                                                   max_df = 0.2,
                                                   sublinear_tf = True)), 
                          ('rfc', RandomForestClassifier(n_estimators=535,
                                                         max_depth=25,
                                                         random_state=42))]
                      )

    model2.fit(X2_train, y2_train)
    y_pred = model2.predict(X2_test)

    print(accuracy_score(y2_test, y_pred))
    print("Number of features:", len(model2.steps[0][1].get_feature_names()))

    # print(model.steps[0][1].get_feature_names())
    # print(model.steps[1][1].feature_importances_)
    # print(model.named_steps['rfc'].feature_importances_)



    feature_importances = pd.DataFrame(model2.steps[1][1].feature_importances_, model2.steps[0][1].get_feature_names()).reset_index()
    feature_importances.columns = ['feature', 'importance']
    features = feature_importances.sort_values('importance', ascending=False)
    print(features.sort_values('importance', ascending=False).head(20))

In [41]:
RandomForest2()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


0.672514619883041
Number of features: 150
               feature  importance
16    business analyst    0.058656
21               cloud    0.021941
15            building    0.016576
17    business process    0.014053
7                agile    0.011957
124           security    0.011854
122               sale    0.011624
83        intelligence    0.011509
59   financial service    0.011383
141               user    0.011356
52               excel    0.011019
128                sql    0.009978
107           platform    0.009887
68                 get    0.009082
86                lead    0.009018
136            testing    0.008298
137           training    0.008270
112            program    0.008167
130           standard    0.008165
80          innovative    0.007963
